<a href="https://colab.research.google.com/github/trongthuan205/visualize_mmdetection/blob/master/%5Bmmdetection%5Dvisualize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install MMDetection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4 MB 1.2 kB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.5.1+cu101 which is incompatible.


In [4]:
# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

     |████████████████████████████████| 307 kB 4.2 MB/s 
     |████████████████████████████████| 185 kB 69.5 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.11-cp37-cp37m-linux_x86_64.whl size=25750859 sha256=19ff8aa809744a8ce9f87389f6ec37d11420ea79fcea325c8c68d5d0d7fa3015
  Stored in directory: /root/.cache/pip/wheels/4c/8f/1d/903456a291e5bf33d99cb03cb1bbc822e2c5d32c123b873ebe
Successfully built mmcv-full


In [15]:
%cd /content/drive/MyDrive/UIT/Research/DIU/src/mmdetection

/content/drive/MyDrive/UIT/Research/DIU/src/mmdetection


In [ ]:
# # # Install mmdetection
# !rm -rf mmdetection
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection

In [5]:
!pip install -e .

ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content


In [6]:
# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

     |████████████████████████████████| 2.1 MB 4.3 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 2.3.3 requires pillow>=7.1.0, but you have pillow 7.0.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [9]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
2.15.0
11.0
GCC 7.5


## Exporting image prediction

In [17]:
! pip install terminaltables

  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=941515fc73c3af8219ff7771c5ec99b71b6aae4865bcde20d92efab38a04957f
  Stored in directory: /root/.cache/pip/wheels/ba/ad/c8/2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built terminaltables


In [18]:
import warnings
warnings.filterwarnings("ignore")
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import os.path
from mmdet.models.detectors import BaseDetector
#BaseDetector.show_result instead of show_result
#import os.path as osp
import pickle
import shutil
import tempfile
import time
from PIL import Image


import mmcv
import torch
import torch.distributed as dist
from mmcv.runner import get_dist_info
from numpy import asarray
from mmcv.utils import is_str
from tqdm.notebook import tqdm

In [19]:
#from mmdet.core import encode_mask_results, tensor2imgs
config_fname = '/content/drive/MyDrive/UIT/Research/FDSE2021/htc_hrnet_dynamic/htc_hrnet_dynamic.py'
checkpoint_fname = '/content/drive/MyDrive/UIT/Research/FDSE2021/htc_hrnet_dynamic/best_bbox_mAP_epoch_17.pth'
model = init_detector(config_fname, checkpoint_fname)

Use load_from_local loader


In [20]:
import os
imgFolderPath = "/content/drive/MyDrive/UIT/Research/DIU/data/CTU_2021/"
outputFolderPath = "CTU_2021_prediction/"
if not os.path.exists(outputFolderPath):
    os.makedirs(outputFolderPath)

In [21]:
for file in tqdm(os.listdir(imgFolderPath)):
  if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".JPG"):
    img = os.path.join(imgFolderPath, file)
    result = inference_detector(model, img)
    model.show_result(img, result, out_file= outputFolderPath + file)
 

  0%|          | 0/14519 [00:00<?, ?it/s]

# Convert ouput prediction to VOC format

In [25]:
! pip install pyqt5

     |████████████████████████████████| 8.3 MB 4.1 MB/s 
     |████████████████████████████████| 317 kB 83.2 MB/s 
     |████████████████████████████████| 59.9 MB 65 kB/s 


In [26]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
import sys
from xml.etree import ElementTree
from xml.etree.ElementTree import Element, SubElement
from lxml import etree
import codecs

XML_EXT = '.xml'
ENCODE_METHOD = 'utf-8'

class PascalVocWriter:

    def __init__(self, foldername, filename, imgSize,databaseSrc='Unknown', localImgPath=None):
        self.foldername = foldername
        self.filename = filename
        self.databaseSrc = databaseSrc
        self.imgSize = imgSize
        self.boxlist = []
        self.localImgPath = localImgPath
        self.verified = False

    def prettify(self, elem):
        """
            Return a pretty-printed XML string for the Element.
        """
        rough_string = ElementTree.tostring(elem, 'utf8')
        root = etree.fromstring(rough_string)
        return etree.tostring(root, pretty_print=True, encoding=ENCODE_METHOD).replace("  ".encode(), "\t".encode())
        # minidom does not support UTF-8
        '''reparsed = minidom.parseString(rough_string)
        return reparsed.toprettyxml(indent="\t", encoding=ENCODE_METHOD)'''

    def genXML(self):
        """
            Return XML root
        """
        # Check conditions
        if self.filename is None or \
                self.foldername is None or \
                self.imgSize is None:
            return None

        top = Element('annotation')
        if self.verified:
            top.set('verified', 'yes')

        folder = SubElement(top, 'folder')
        folder.text = self.foldername

        filename = SubElement(top, 'filename')
        filename.text = self.filename

        if self.localImgPath is not None:
            localImgPath = SubElement(top, 'path')
            localImgPath.text = self.localImgPath

        source = SubElement(top, 'source')
        database = SubElement(source, 'database')
        database.text = self.databaseSrc

        size_part = SubElement(top, 'size')
        width = SubElement(size_part, 'width')
        height = SubElement(size_part, 'height')
        depth = SubElement(size_part, 'depth')
        width.text = str(self.imgSize[1])
        height.text = str(self.imgSize[0])
        if len(self.imgSize) == 3:
            depth.text = str(self.imgSize[2])
        else:
            depth.text = '1'

        segmented = SubElement(top, 'segmented')
        segmented.text = '0'
        return top

    def addBndBox(self, xmin, ymin, xmax, ymax, name, difficult):
        bndbox = {'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax}
        bndbox['name'] = name
        bndbox['difficult'] = difficult
        self.boxlist.append(bndbox)

    def appendObjects(self, top):
        for each_object in self.boxlist:
            object_item = SubElement(top, 'object')
            name = SubElement(object_item, 'name')
            try:
                name.text = unicode(each_object['name'])
            except NameError:
                # Py3: NameError: name 'unicode' is not defined
                name.text = each_object['name']
            pose = SubElement(object_item, 'pose')
            pose.text = "Unspecified"
            truncated = SubElement(object_item, 'truncated')
            if int(each_object['ymax']) == int(self.imgSize[0]) or (int(each_object['ymin'])== 1):
                truncated.text = "1" # max == height or min
            elif (int(each_object['xmax'])==int(self.imgSize[1])) or (int(each_object['xmin'])== 1):
                truncated.text = "1" # max == width or min
            else:
                truncated.text = "0"
            difficult = SubElement(object_item, 'difficult')
            difficult.text = str( bool(each_object['difficult']) & 1 )
            bndbox = SubElement(object_item, 'bndbox')
            xmin = SubElement(bndbox, 'xmin')
            xmin.text = str(each_object['xmin'])
            ymin = SubElement(bndbox, 'ymin')
            ymin.text = str(each_object['ymin'])
            xmax = SubElement(bndbox, 'xmax')
            xmax.text = str(each_object['xmax'])
            ymax = SubElement(bndbox, 'ymax')
            ymax.text = str(each_object['ymax'])

    def save(self, targetFile=None):
        root = self.genXML()
        self.appendObjects(root)
        out_file = None
        if targetFile is None:
            out_file = codecs.open(
                self.filename + XML_EXT, 'w', encoding=ENCODE_METHOD)
        else:
            out_file = codecs.open(targetFile, 'w', encoding=ENCODE_METHOD)

        prettifyResult = self.prettify(root)
        out_file.write(prettifyResult.decode('utf8'))
        out_file.close()


class PascalVocReader:

    def __init__(self, filepath):
        # shapes type:
        # [labbel, [(x1,y1), (x2,y2), (x3,y3), (x4,y4)], color, color, difficult]
        self.shapes = []
        self.filepath = filepath
        self.verified = False
        try:
            self.parseXML()
        except:
            pass

    def getShapes(self):
        return self.shapes

    def addShape(self, label, bndbox, difficult):
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        points = [(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)]
        self.shapes.append((label, points, None, None, difficult))

    def parseXML(self):
        assert self.filepath.endswith(XML_EXT), "Unsupport file format"
        parser = etree.XMLParser(encoding=ENCODE_METHOD)
        xmltree = ElementTree.parse(self.filepath, parser=parser).getroot()
        filename = xmltree.find('filename').text
        try:
            verified = xmltree.attrib['verified']
            if verified == 'yes':
                self.verified = True
        except KeyError:
            self.verified = False

        for object_iter in xmltree.findall('object'):
            bndbox = object_iter.find("bndbox")
            label = object_iter.find('name').text
            # Add chris
            difficult = False
            if object_iter.find('difficult') is not None:
                difficult = bool(int(object_iter.find('difficult').text))
            self.addShape(label, bndbox, difficult)
        return True

In [28]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
import sys
import os
from xml.etree import ElementTree
from xml.etree.ElementTree import Element, SubElement
from lxml import etree
import codecs
# from constants import DEFAULT_ENCODING

TXT_EXT = '.txt'
ENCODE_METHOD = 'utf-8'
# ENCODE_METHOD = DEFAULT_ENCODING

class YOLOWriter:

    def __init__(self, foldername, filename, imgSize, databaseSrc='Unknown', localImgPath=None):
        self.foldername = foldername
        self.filename = filename
        self.databaseSrc = databaseSrc
        self.imgSize = imgSize
        self.boxlist = []
        self.localImgPath = localImgPath
        self.verified = False

    def addBndBox(self, xmin, ymin, xmax, ymax, name, difficult):
        bndbox = {'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax}
        bndbox['name'] = name
        bndbox['difficult'] = difficult
        self.boxlist.append(bndbox)

    def BndBox2YoloLine(self, box, classList=[]):
        xmin = box['xmin']
        xmax = box['xmax']
        ymin = box['ymin']
        ymax = box['ymax']

        xcen = float((xmin + xmax)) / 2 / self.imgSize[1]
        ycen = float((ymin + ymax)) / 2 / self.imgSize[0]

        w = float((xmax - xmin)) / self.imgSize[1]
        h = float((ymax - ymin)) / self.imgSize[0]

        # PR387
        boxName = box['name']
        if boxName not in classList:
            classList.append(boxName)

        classIndex = classList.index(boxName)

        return classIndex, xcen, ycen, w, h

    def save(self, classList=[], targetFile=None):

        out_file = None #Update yolo .txt
        out_class_file = None   #Update class list .txt

        if targetFile is None:
            out_file = open(
            self.filename + TXT_EXT, 'w', encoding=ENCODE_METHOD)
            classesFile = os.path.join(os.path.dirname(os.path.abspath(self.filename)), "classes.txt")
            out_class_file = open(classesFile, 'w')

        else:
            out_file = codecs.open(targetFile, 'w', encoding=ENCODE_METHOD)
            classesFile = os.path.join(os.path.dirname(os.path.abspath(targetFile)), "classes.txt")
            out_class_file = open(classesFile, 'w')


        for box in self.boxlist:
            classIndex, xcen, ycen, w, h = self.BndBox2YoloLine(box, classList)
            # print (classIndex, xcen, ycen, w, h)
            out_file.write("%d %.6f %.6f %.6f %.6f\n" % (classIndex, xcen, ycen, w, h))

        # print (classList)
        # print (out_class_file)
        for c in classList:
            out_class_file.write(c+'\n')

        out_class_file.close()
        out_file.close()



class YoloReader:

    def __init__(self, filepath, image, classListPath=None):
        # shapes type:
        # [labbel, [(x1,y1), (x2,y2), (x3,y3), (x4,y4)], color, color, difficult]
        self.shapes = []
        self.filepath = filepath

        if classListPath is None:
            dir_path = os.path.dirname(os.path.realpath(self.filepath))
            self.classListPath = os.path.join(dir_path, "classes.txt")
        else:
            self.classListPath = classListPath

        # print (filepath, self.classListPath)

        classesFile = open(self.classListPath, 'r')
        self.classes = classesFile.read().strip('\n').split('\n')

        # print (self.classes)

        imgSize = [image.height(), image.width(),
                      1 if image.isGrayscale() else 3]

        self.imgSize = imgSize

        self.verified = False
        # try:
        self.parseYoloFormat()
        # except:
            # pass

    def getShapes(self):
        return self.shapes

    def addShape(self, label, xmin, ymin, xmax, ymax, difficult):

        points = [(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)]
        self.shapes.append((label, points, None, None, difficult))

    def getLabel(self, classIndex):
        label = self.classes[int(classIndex)]
        return label

    def yoloLine2Shape(self, classIndex, xcen, ycen, w, h):
        label = self.classes[int(classIndex)]

#        xmin = max(float(xcen) - float(w) / 2, 0)
#        xmax = min(float(xcen) + float(w) / 2, 1)
#        ymin = max(float(ycen) - float(h) / 2, 0)
#        ymax = min(float(ycen) + float(h) / 2, 1)
#
#        xmin = int(self.imgSize[1] * xmin)
#        xmax = int(self.imgSize[1] * xmax)
#        ymin = int(self.imgSize[0] * ymin)
#        ymax = int(self.imgSize[0] * ymax)
        
        xmin = int(xcen)
        xmax = int(w)
        ymin = int(ycen)
        ymax = int(h)

        return label, xmin, ymin, xmax, ymax

    def parseYoloFormat(self):
        bndBoxFile = open(self.filepath, 'r')
        for bndBox in bndBoxFile:
            classIndex, xcen, ycen, w, h = bndBox.split(' ')
            label, xmin, ymin, xmax, ymax = self.yoloLine2Shape(classIndex, xcen, ycen, w, h)

            # Caveat: difficult flag is discarded when saved as yolo format.
            self.addShape(label, xmin, ymin, xmax, ymax, False)


In [29]:
# from pascal_voc_io import XML_EXT
# from tqdm import tqdm
# from pascal_voc_io import PascalVocWriter
# from yolo_io import YoloReader
import os.path
import sys
from PyQt5.QtGui import QImage
# try:
#     from PyQt5.QtGui import QImage
# except ImportError:
#     from PyQt4.QtGui import QImage

In [54]:
imgFolderPath = "CTU_2021_prediction/"
VOCFolderpath = "VOC_annot/"
if not os.path.exists(VOCFolderpath):
    os.makedirs(VOCFolderpath)

- Tạo file classes.txt bao gồm tên các class theo đúng thứ tự (tên mỗi class trên một dòng)
- Lưu trong cùng thư mục với thư mục chứa các file txt.

In [57]:
# Search all yolo annotation (txt files) in this folder
for file in tqdm(os.listdir(imgFolderPath)):
    if file.endswith(".txt") and file != "classes.txt":

        annotation_no_txt = os.path.splitext(file)[0]

        imagePath = imgFolderPath + "/" + annotation_no_txt + ".jpg"
        if not os.path.isfile(imagePath):
            imagePath = imgFolderPath + "/" + annotation_no_txt + ".jpeg"
            if not os.path.isfile(imagePath):
              imagePath = imgFolderPath + "/" + annotation_no_txt + ".JPG"

        image = QImage()
        image.load(imagePath)
        imageShape = [image.height(), image.width(), 1 if image.isGrayscale() else 3]
        imgFolderName = os.path.basename(imgFolderPath)
        imgFileName = os.path.basename(imagePath)

        writer = PascalVocWriter(imgFolderName, imgFileName, imageShape, localImgPath=imagePath)


        # Read YOLO file
        txtPath = imgFolderPath + "/" + file
        tYoloParseReader = YoloReader(txtPath, image)
        shapes = tYoloParseReader.getShapes()
        num_of_box = len(shapes)

        for i in range(num_of_box):
            label = shapes[i][0]
            xmin = shapes[i][1][0][0]
            ymin = shapes[i][1][0][1]
            x_max = shapes[i][1][2][0]
            y_max = shapes[i][1][2][1]

            writer.addBndBox(xmin, ymin, x_max, y_max, label, 0)

        writer.save(targetFile= VOCFolderpath + "/" + annotation_no_txt + ".xml")


  0%|          | 0/29040 [00:00<?, ?it/s]

# Visualize Ground-Truth vs Predict in class

In [ ]:
import pandas as pd
import numpy as np
def read_ann_pred(file):
    filesize = os.path.getsize(file)
    if filesize == 0:
        return []
    else:
        ann = pd.read_csv(file, header = None, sep = ' ')
        ann = ann[ann[0] == 3]
        return np.array(ann.iloc[:,1:])

In [ ]:
import xml.etree.ElementTree as ET
def read_ann_gt(file):
    ann = []
    tree = ET.parse(file)
    root = tree.getroot()
    for element in root:
        if (element.tag=='object'):
            if element[0].text == "formula":
                ann.append([int(element[4][0].text), int(element[4][1].text),
                        int(element[4][2].text), int(element[4][3].text)])
    return ann

In [ ]:
from PIL import Image
from PIL import Image
from PIL import ImageFont, ImageDraw
def draw_bbox(image, gts, preds, save_folder):
    img = Image.open(image)
    if img.mode == 'L':
        img = img.convert('RGB')
    draw = ImageDraw.Draw(img, 'RGBA')
    if len(preds) > 0:
        for pred in preds:
            draw.rectangle((pred[0], pred[1], pred[2], pred[3]),
                        outline = (255, 0, 0), width = 5)
    if len(gts) > 0:
        for gt in gts:
            draw.rectangle((gt[0], gt[1], gt[2], gt[3]),
                        outline = (0, 255, 0),width = 5)
    out_file = os.path.join(save_folder, image.split('/')[-1])
    img.save(out_file)

In [ ]:
for i in tqdm(os.listdir('imgs')):
    img = os.path.join('imgs', i)

    gt_file = (img.split('.')[0] + '.xml')
    gt_file = gt_file.split('/')[-1]

    pred_file = (img.split('.')[0] + '.txt')
    pred_file = pred_file.split('/')[-1]



    gt = read_ann_gt(os.path.join('gt', gt_file))
    pred = read_ann_pred(os.path.join('predict_cascade', pred_file))

    save = 'formula_cascade'

    draw_bbox(img, gt, pred, save)

In [ ]:
for i in tqdm(os.listdir('imgs')):
    img = os.path.join('imgs', i)

    gt_file = (img.split('.')[0] + '.xml')
    gt_file = gt_file.split('/')[-1]
    pred_file = (img.split('.')[0] + '.txt')
    pred_file = pred_file.split('/')[-1]

    gt = read_ann_gt(os.path.join('gt', gt_file))
    pred = read_ann_pred(os.path.join('predict_faster', pred_file))

    save = 'formula_faster'

    draw_bbox(img, gt, pred, save)